In [1]:
#actionComment  추출 개선
# drafter, createdAt, activities만 UPDATE하는 버전
# 테이블 기반 추출 + CSV 조직도 매칭 + 결재의견 추가 + KST 시간대 처리
"""
 [기능]
 HTML 파일에서 drafter, createdAt, activities를 재추출하여
 DB documents 테이블의 해당 컬럼만 UPDATE
 (테이블 기반 추출 + CSV 조직도 매칭 + 결재의견 추가)

 [추출 대상]
 - drafter: 기안자 정보 (테이블의 "기안자" 행에서)
 - createdAt: 기안일시 (테이블의 "기안일" 행에서, KST→Unix ms)
 - activities: 결재 활동 로그 (bg02 div에서)
   - actionComment: user_spans에서 추가 추출

 [CSV 조직도 매칭]
 - 현직자: CSV에서 positionName, deptName, emailId, deptCode 매칭
 - 퇴사자: emailId를 'master'로 설정, 나머지 공란

 [activities 추출 로직]
 1. bg02 div 내 ul > li에서 이름, 타입, 날짜 추출
 2. 타입 결정: 기안→DRAFT, 합의→AGREEMENT, 그외→APPROVAL
 3. 날짜: MM/DD 형식 → createdAt 연도 + KST 변환
 4. actionComment: user_spans 순회하며 다음 div에서 의견 추출

 [시간대 처리]
 - 모든 날짜를 KST(Asia/Seoul)로 처리 후 Unix timestamp 변환

 [동작 순서]
 1. CSV 조직도 로드
 2. HTML 폴더에서 파일 순회
 3. 각 HTML에서 drafter, createdAt, activities 추출
 4. JSON 파일로 저장
 5. DB UPDATE (source_id + end_year로 매칭)

 [출력]
 - JSON 파일: update_drafter_activities.json
 - MariaDB: documents 테이블의 drafter_*, created_at, activities 컬럼 UPDATE

 [설정 (#여기를 수정하세요)]
 - base_path: HTML 폴더 경로
 - end_year: 대상 연도 (WHERE 조건)
 - csv_file: 인사정보 CSV 파일
 - db_config: DB 접속 정보

 [의존성]
 - beautifulsoup4
 - pymysql
 - pandas
 - pytz
 -> pip install beautifulsoup4 pymysql pandas pytz
"""
import os
import json
import re
from datetime import datetime
import pytz
from bs4 import BeautifulSoup
import pymysql
from pathlib import Path
import pandas as pd

class ApprovalDocParser:
    def __init__(self, base_path, csv_file):
        self.base_path = base_path
        self.kst = pytz.timezone('Asia/Seoul')
        
        # CSV 조직도 로드
        print(f"📊 CSV 조직도 로드 중... ({csv_file})")
        df = pd.read_csv(csv_file, encoding='utf-8-sig')
        self.employee_dict = {}
        for _, row in df.iterrows():
            self.employee_dict[row['사원명']] = {
                'emailId': row['ID'],
                'deptName': row['부서'],
                'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
                'positionName': row['직위'] if pd.notna(row['직위']) else '',
                'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''
            }
        print(f"✅ 총 {len(self.employee_dict)}명 로드\n")
        
    def extract_source_id(self, filename):
        """파일명에서 마지막 숫자 추출"""
        numbers = re.findall(r'\d+', filename)
        return numbers[-1] if numbers else None
    
    def extract_person_info(self, text):
        """이름/직책/부서 형식에서 정보 추출"""
        text = re.sub(r'\d+', '', text).strip()
        parts = text.split('/')
        if len(parts) >= 3:
            return {
                'name': parts[0].strip(),
                'positionName': parts[1].strip(),
                'deptName': parts[2].strip()
            }
        return None
    
    def parse_html(self, html_path):
        """HTML 파일에서 drafter, createdAt, activities만 추출"""
        with open(html_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
        
        filename = os.path.basename(html_path)
        source_id = self.extract_source_id(filename)
        
        # === 1. drafter 추출 (테이블에서) ===
        drafter = {}
        drafter_th = soup.find('th', string=lambda s: s and '기안자' in s)
        if drafter_th:
            drafter_td = drafter_th.find_next_sibling('td')
            if drafter_td:
                bg01_div = drafter_td.find('div', class_='bg01')
                if bg01_div:
                    name = bg01_div.get_text(strip=True)
                    if name:
                        # CSV 조직도 매칭
                        if name in self.employee_dict:
                            drafter = {
                                'name': name,
                                'positionName': self.employee_dict[name]['positionName'],
                                'deptName': self.employee_dict[name]['deptName'],
                                'emailId': self.employee_dict[name]['emailId'],
                                'deptCode': self.employee_dict[name]['deptCode']
                            }
                        else:
                            # 퇴사자
                            drafter = {
                                'name': name,
                                'positionName': '',
                                'deptName': '',
                                'emailId': 'master',
                                'deptCode': ''
                            }
        
        # === 2. createdAt 추출 (테이블에서) - KST 처리 ===
        created_at = None
        created_year = None
        created_th = soup.find('th', string=lambda s: s and '기안일' in s)
        if created_th:
            created_td = created_th.find_next_sibling('td')
            if created_td:
                bg01_div = created_td.find('div', class_='bg01')
                if bg01_div:
                    date_str = bg01_div.get_text(strip=True)
                    try:
                        dt_naive = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
                        dt_kst = self.kst.localize(dt_naive)
                        created_at = int(dt_kst.timestamp() * 1000)
                        created_year = dt_kst.year
                    except:
                        pass
        
        # === 3. activities 추출 (bg02에서) - KST 처리 ===
        activities = []
        bg02_divs = soup.find_all('div', class_='bg02')
        
        for idx, bg02 in enumerate(bg02_divs):
            ul = bg02.find('ul')
            if ul:
                lis = ul.find_all('li')
                if len(lis) >= 2:
                    name = lis[0].get_text(strip=True)
                    action_type_text = lis[1].get_text(strip=True)
                    date_text = lis[2].get_text(strip=True) if len(lis) >= 3 else ''
                    
                    if name:
                        # 타입 결정
                        if '기안' in action_type_text:
                            action_type = 'DRAFT'
                        elif '합의' in action_type_text:
                            action_type = 'AGREEMENT'
                        else:
                            action_type = 'APPROVAL'
                        
                        # 날짜 변환 (00:00:00) - KST 처리
                        action_date = None
                        if date_text and created_year:
                            try:
                                full_date = f"{created_year}-{date_text.replace('/', '-')}"
                                dt_naive = datetime.strptime(full_date, "%Y-%m-%d")
                                dt_kst = self.kst.localize(dt_naive)
                                action_date = int(dt_kst.timestamp() * 1000)
                            except:
                                pass
                        
                        # CSV 조직도 매칭
                        if name in self.employee_dict:
                            position = self.employee_dict[name]['positionName']
                            dept = self.employee_dict[name]['deptName']
                            email = self.employee_dict[name]['emailId']
                            dept_code = self.employee_dict[name]['deptCode']
                        else:
                            # 퇴사자
                            position = ''
                            dept = ''
                            email = ''
                            dept_code = ''
                        
                        activities.append({
                            'positionName': position,
                            'deptName': dept,
                            'actionLogType': action_type,
                            'name': name,
                            'emailId': email,
                            'type': action_type,
                            'actionDate': action_date,
                            'deptCode': dept_code,
                            'actionComment': ''  # 일단 빈값
                        })
        
        # === 4. actionComment 추가 (user_spans에서) - 개선됨 ===
        user_spans = soup.find_all('span', class_='user')
        for user_span in user_spans:
            name_elem = user_span.find('span', class_='F_12_black_b')
            if name_elem:
                info = self.extract_person_info(name_elem.get_text(strip=True))
                if info:
                    name = info['name']
                    
                    # 의견 추출 - user_span 이후 다음 user_span 전까지의 div 찾기
                    action_comment = ""
                    
                    # next_siblings를 사용하여 순차 탐색
                    for sibling in user_span.next_siblings:
                        # 다음 user_span을 만나면 중단
                        if hasattr(sibling, 'name'):
                            if sibling.name == 'span' and 'user' in sibling.get('class', []):
                                break
                            # div를 찾으면 저장하고 중단
                            if sibling.name == 'div':
                                action_comment = sibling.get_text(strip=True)
                                break
                    
                    # activities에서 이름 찾아서 의견 추가
                    for activity in activities:
                        if activity['name'] == name:
                            activity['actionComment'] = action_comment
                            break
        
        # 결과 반환 (필요한 3가지만)
        return {
            'sourceId': source_id,
            'drafter': drafter,
            'createdAt': created_at,
            'activities': activities
        }
    
    def process_all_files(self):
        """모든 HTML 파일 처리"""
        all_results = []
        approval_path = Path(self.base_path) / '결재'
        
        if not approval_path.exists():
            print(f"경로를 찾을 수 없습니다: {approval_path}")
            return all_results
            
        html_files = list(approval_path.rglob('*.html'))
        print(f"총 {len(html_files)}개의 HTML 파일을 찾았습니다.")
        
        for idx, html_file in enumerate(html_files, 1):
            try:
                if idx % 100 == 0 or idx == len(html_files):
                    print(f"처리 중... [{idx}/{len(html_files)}] {html_file.name}")
                result = self.parse_html(html_file)
                all_results.append(result)
            except Exception as e:
                print(f"오류 발생 ({html_file.name}): {e}")
        
        return all_results
    
    def save_to_json(self, data, output_path):
        """JSON 파일로 저장"""
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"JSON 파일 저장 완료: {output_path}")
    
    def save_to_mariadb_update(self, data, db_config, end_year):
        """MariaDB 특정 컬럼만 UPDATE"""
        conn = None
        cursor = None
        
        try:
            print("\n=== DB 연결 시작 ===")
            print(f"Host: {db_config['host']}, Database: {db_config['database']}")
            
            # DB 연결 - FOUND_ROWS 플래그 추가
            conn = pymysql.connect(
                host=db_config['host'],
                user=db_config['user'],
                password=db_config['password'],
                database=db_config['database'],
                charset='utf8mb4',
                client_flag=pymysql.constants.CLIENT.FOUND_ROWS
            )
            print("✓ DB 연결 성공 (FOUND_ROWS 모드)")
            cursor = conn.cursor()
            
            # UPDATE 쿼리
            update_sql = """
            UPDATE documents 
            SET drafter_name = %s,
                drafter_position = %s,
                drafter_dept = %s,
                created_at = %s,
                activities = %s
            WHERE source_id = %s AND end_year = %s
            """
            
            success_count = 0
            not_found_count = 0
            error_count = 0
            error_details = []
            not_found_ids = []
            
            print(f"\n=== 데이터 업데이트 시작 ({len(data)}건) ===")
            
            for idx, doc in enumerate(data, 1):
                try:
                    def safe_json(value):
                        if not value:
                            return '[]'
                        try:
                            return json.dumps(value, ensure_ascii=False)
                        except:
                            return '[]'
                    
                    source_id = doc.get('sourceId')
                    
                    values = (
                        doc.get('drafter', {}).get('name', ''),
                        doc.get('drafter', {}).get('positionName', ''),
                        doc.get('drafter', {}).get('deptName', ''),
                        doc.get('createdAt'),
                        safe_json(doc.get('activities', [])),
                        source_id,
                        end_year
                    )
                    
                    cursor.execute(update_sql, values)
                    
                    # FOUND_ROWS 모드: 매칭된 행 수 반환
                    if cursor.rowcount > 0:
                        success_count += 1
                    else:
                        not_found_count += 1
                        not_found_ids.append(source_id)
                    
                    if idx % 100 == 0 or idx == len(data):
                        print(f"  진행: {idx}/{len(data)} (매칭: {success_count}, 없음: {not_found_count}, 실패: {error_count})")
                    
                except Exception as e:
                    error_count += 1
                    error_msg = f"sourceId: {doc.get('sourceId')} - {str(e)[:80]}"
                    error_details.append(error_msg)
                    if error_count <= 5:
                        print(f"  [오류 {error_count}] {error_msg}")
            
            print("\n=== 커밋 시작 ===")
            conn.commit()
            print("✓ 커밋 완료")
            
            print(f"\n=== 결과 요약 ===")
            print(f"✓ 매칭 성공: {success_count}건")
            print(f"⚠ DB에 없음: {not_found_count}건")
            print(f"✗ 오류: {error_count}건")
            
            if not_found_ids:
                print(f"\n⚠ DB에서 못 찾은 source_id 샘플 (처음 20개):")
                for nf_id in not_found_ids[:20]:
                    print(f"  - {nf_id}")
                
                cursor.execute("""
                    SELECT MIN(source_id), MAX(source_id) 
                    FROM documents 
                    WHERE end_year = %s
                """, (end_year,))
                min_id, max_id = cursor.fetchone()
                print(f"\n📊 DB의 source_id 범위: {min_id} ~ {max_id}")
            
            if error_count > 5:
                print(f"\n처음 5개 외 {error_count - 5}개의 추가 오류가 발생했습니다.")
                print("모든 오류 보기:")
                for err in error_details[:20]:
                    print(f"  - {err}")
                if len(error_details) > 20:
                    print(f"  ... 외 {len(error_details) - 20}개 더")
            
        except pymysql.Error as e:
            print(f"\n❌ DB 오류 발생:")
            print(f"  Error Code: {e.args[0]}")
            print(f"  Error Message: {e.args[1]}")
            
        except Exception as e:
            print(f"\n❌ 일반 오류 발생: {e}")
            
        finally:
            if cursor:
                cursor.close()
            if conn:
                conn.close()
                print("\n✓ DB 연결 종료")

#여기를 수정하세요
def main():
    base_path = r'C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html'
    end_year = 2025
    csv_file = '인사정보_부서코드추가.csv'
    
    parser = ApprovalDocParser(base_path, csv_file)
    
    print("HTML 파일 파싱 시작...")
    results = parser.process_all_files()
    
    output_json_path = 'update_drafter_activities.json'
    parser.save_to_json(results, output_json_path)
    
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '1234',
        'database': 'any_approval'
    }
    
    print("\n" + "="*50)
    parser.save_to_mariadb_update(results, db_config, end_year)
    print("="*50)
    
    print(f"\n완료! 총 {len(results)}건 처리됨")


if __name__ == '__main__':
    main()

📊 CSV 조직도 로드 중... (인사정보_부서코드추가.csv)
✅ 총 156명 로드

HTML 파일 파싱 시작...
총 8175개의 HTML 파일을 찾았습니다.
처리 중... [100/8175] 20210204_명함발급을 요청합니다._15362351.html
처리 중... [200/8175] 20210323_연차 품의서_15618962.html
처리 중... [300/8175] 20210504_연차휴가 신청_15873929.html
처리 중... [400/8175] 20210611_2021-06-14(월) 휴가 신청합니다 - 1일_16094756.html
처리 중... [500/8175] 20210707_오전 반차 신청합니다._16246418.html
처리 중... [600/8175] 20210728_김상욱 휴가 사용 신청서 입니다._16375259.html
처리 중... [700/8175] 20210820_08-16 NHN NB, Monitor x 23_16509307.html
처리 중... [800/8175] 20210914_2021년 9월 16일(목) 연차 휴가(백신) 신청합니다_16657522.html
처리 중... [900/8175] 20211008_[구매_외주인력] (주)엘지화학_[R&amp;D] LG화학 서비스설명서_특허시스템_SM운영 - 김광림(프리랜서)_16782949.html
처리 중... [1000/8175] 20211028_10월 기타 경비 신청합니다_16906849.html
처리 중... [1100/8175] 20211116_11-18 MHNco NB x 1_17024023.html
처리 중... [1200/8175] 20211210_(기능개선) OCI 특허관리시스템 조사분석 및 ERP 기능개선 계약품의_17187276.html
처리 중... [1300/8175] 20211224_휴가 사용 신청서_이의환 선임_17276280.html
처리 중... [1400/8175] 20220113_휴가 사용 신청합니다._17403800.html
처

In [5]:
# drafter, createdAt, activities만 UPDATE하는 버전
# 테이블 기반 추출 + CSV 조직도 매칭 + 결재의견 추가 + KST 시간대 처리

import os
import json
import re
from datetime import datetime
import pytz  # ← 추가
from bs4 import BeautifulSoup
import pymysql
from pathlib import Path
import pandas as pd

class ApprovalDocParser:
    def __init__(self, base_path, csv_file):
        self.base_path = base_path
        self.kst = pytz.timezone('Asia/Seoul')  # ← 추가: 한국 시간대
        
        # CSV 조직도 로드
        print(f"📊 CSV 조직도 로드 중... ({csv_file})")
        df = pd.read_csv(csv_file, encoding='utf-8-sig')
        self.employee_dict = {}
        for _, row in df.iterrows():
            self.employee_dict[row['사원명']] = {
                'emailId': row['ID'],
                'deptName': row['부서'],
                'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
                'positionName': row['직위'] if pd.notna(row['직위']) else '',
                'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''
            }
        print(f"✅ 총 {len(self.employee_dict)}명 로드\n")
        
    def extract_source_id(self, filename):
        """파일명에서 마지막 숫자 추출"""
        numbers = re.findall(r'\d+', filename)
        return numbers[-1] if numbers else None
    
    def extract_person_info(self, text):
        """이름/직책/부서 형식에서 정보 추출"""
        text = re.sub(r'\d+', '', text).strip()
        parts = text.split('/')
        if len(parts) >= 3:
            return {
                'name': parts[0].strip(),
                'positionName': parts[1].strip(),
                'deptName': parts[2].strip()
            }
        return None
    
    def parse_html(self, html_path):
        """HTML 파일에서 drafter, createdAt, activities만 추출"""
        with open(html_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
        
        filename = os.path.basename(html_path)
        source_id = self.extract_source_id(filename)
        
        # === 1. drafter 추출 (테이블에서) ===
        drafter = {}
        drafter_th = soup.find('th', string=lambda s: s and '기안자' in s)
        if drafter_th:
            drafter_td = drafter_th.find_next_sibling('td')
            if drafter_td:
                bg01_div = drafter_td.find('div', class_='bg01')
                if bg01_div:
                    name = bg01_div.get_text(strip=True)
                    if name:
                        # CSV 조직도 매칭
                        if name in self.employee_dict:
                            drafter = {
                                'name': name,
                                'positionName': self.employee_dict[name]['positionName'],
                                'deptName': self.employee_dict[name]['deptName'],
                                'emailId': self.employee_dict[name]['emailId'],
                                'deptCode': self.employee_dict[name]['deptCode']
                            }
                        else:
                            # 퇴사자
                            drafter = {
                                'name': name,
                                'positionName': '',
                                'deptName': '',
                                'emailId': 'master',
                                'deptCode': ''
                            }
        
        # === 2. createdAt 추출 (테이블에서) - KST 처리 추가 ===
        created_at = None
        created_year = None
        created_th = soup.find('th', string=lambda s: s and '기안일' in s)
        if created_th:
            created_td = created_th.find_next_sibling('td')
            if created_td:
                bg01_div = created_td.find('div', class_='bg01')
                if bg01_div:
                    date_str = bg01_div.get_text(strip=True)
                    try:
                        # naive datetime 생성 후 KST로 명시
                        dt_naive = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
                        dt_kst = self.kst.localize(dt_naive)
                        created_at = int(dt_kst.timestamp() * 1000)
                        created_year = dt_kst.year
                    except:
                        pass
        
        # === 3. activities 추출 (bg02에서) - KST 처리 추가 ===
        activities = []
        bg02_divs = soup.find_all('div', class_='bg02')
        
        for idx, bg02 in enumerate(bg02_divs):
            ul = bg02.find('ul')
            if ul:
                lis = ul.find_all('li')
                if len(lis) >= 2:
                    name = lis[0].get_text(strip=True)
                    action_type_text = lis[1].get_text(strip=True)
                    date_text = lis[2].get_text(strip=True) if len(lis) >= 3 else ''
                    
                    if name:
                        # 타입 결정
                        if '기안' in action_type_text:
                            action_type = 'DRAFT'
                        elif '합의' in action_type_text:
                            action_type = 'AGREEMENT'
                        else:
                            action_type = 'APPROVAL'
                        
                        # 날짜 변환 (00:00:00) - KST 처리 추가
                        action_date = None
                        if date_text and created_year:
                            try:
                                full_date = f"{created_year}-{date_text.replace('/', '-')}"
                                dt_naive = datetime.strptime(full_date, "%Y-%m-%d")
                                dt_kst = self.kst.localize(dt_naive)
                                action_date = int(dt_kst.timestamp() * 1000)
                            except:
                                pass
                        
                        # CSV 조직도 매칭
                        if name in self.employee_dict:
                            position = self.employee_dict[name]['positionName']
                            dept = self.employee_dict[name]['deptName']
                            email = self.employee_dict[name]['emailId']
                            dept_code = self.employee_dict[name]['deptCode']
                        else:
                            # 퇴사자
                            position = ''
                            dept = ''
                            email = ''
                            dept_code = ''
                        
                        activities.append({
                            'positionName': position,
                            'deptName': dept,
                            'actionLogType': action_type,
                            'name': name,
                            'emailId': email,
                            'type': action_type,
                            'actionDate': action_date,
                            'deptCode': dept_code,
                            'actionComment': ''  # 일단 빈값
                        })
        
        # === 4. actionComment 추가 (user_spans에서) ===
        user_spans = soup.find_all('span', class_='user')
        for user_span in user_spans:
            name_elem = user_span.find('span', class_='F_12_black_b')
            if name_elem:
                info = self.extract_person_info(name_elem.get_text(strip=True))
                if info:
                    name = info['name']
                    
                    # 의견 추출 - user_span의 부모(td)에서 div 찾기
                    action_comment = ""
                    parent = user_span.parent
                    if parent:
                        comment_div = parent.find('div')
                        if comment_div:
                            action_comment = comment_div.get_text(strip=True)
                    
                    # activities에서 이름 찾아서 의견 추가
                    for activity in activities:
                        if activity['name'] == name:
                            activity['actionComment'] = action_comment
                            break
        
        # 결과 반환 (필요한 3가지만)
        return {
            'sourceId': source_id,
            'drafter': drafter,
            'createdAt': created_at,
            'activities': activities
        }
    
    def process_all_files(self):
        """모든 HTML 파일 처리"""
        all_results = []
        approval_path = Path(self.base_path) / '결재'
        
        if not approval_path.exists():
            print(f"경로를 찾을 수 없습니다: {approval_path}")
            return all_results
            
        html_files = list(approval_path.rglob('*.html'))
        print(f"총 {len(html_files)}개의 HTML 파일을 찾았습니다.")
        
        for idx, html_file in enumerate(html_files, 1):
            try:
                if idx % 100 == 0 or idx == len(html_files):
                    print(f"처리 중... [{idx}/{len(html_files)}] {html_file.name}")
                result = self.parse_html(html_file)
                all_results.append(result)
            except Exception as e:
                print(f"오류 발생 ({html_file.name}): {e}")
        
        return all_results
    
    def save_to_json(self, data, output_path):
        """JSON 파일로 저장"""
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"JSON 파일 저장 완료: {output_path}")
    
    def save_to_mariadb_update(self, data, db_config, end_year):
        """MariaDB 특정 컬럼만 UPDATE"""
        conn = None
        cursor = None
        
        try:
            print("\n=== DB 연결 시작 ===")
            print(f"Host: {db_config['host']}, Database: {db_config['database']}")
            
            # DB 연결 - FOUND_ROWS 플래그 추가
            conn = pymysql.connect(
                host=db_config['host'],
                user=db_config['user'],
                password=db_config['password'],
                database=db_config['database'],
                charset='utf8mb4',
                client_flag=pymysql.constants.CLIENT.FOUND_ROWS  # ← 추가!
            )
            print("✓ DB 연결 성공 (FOUND_ROWS 모드)")
            cursor = conn.cursor()
            
            # UPDATE 쿼리
            update_sql = """
            UPDATE documents 
            SET drafter_name = %s,
                drafter_position = %s,
                drafter_dept = %s,
                created_at = %s,
                activities = %s
            WHERE source_id = %s AND end_year = %s
            """
            
            success_count = 0
            not_found_count = 0
            error_count = 0
            error_details = []
            not_found_ids = []
            
            print(f"\n=== 데이터 업데이트 시작 ({len(data)}건) ===")
            
            for idx, doc in enumerate(data, 1):
                try:
                    def safe_json(value):
                        if not value:
                            return '[]'
                        try:
                            return json.dumps(value, ensure_ascii=False)
                        except:
                            return '[]'
                    
                    source_id = doc.get('sourceId')
                    
                    values = (
                        doc.get('drafter', {}).get('name', ''),
                        doc.get('drafter', {}).get('positionName', ''),
                        doc.get('drafter', {}).get('deptName', ''),
                        doc.get('createdAt'),
                        safe_json(doc.get('activities', [])),
                        source_id,
                        end_year
                    )
                    
                    cursor.execute(update_sql, values)
                    
                    # FOUND_ROWS 모드: 매칭된 행 수 반환 (변경 여부 무관)
                    if cursor.rowcount > 0:
                        success_count += 1
                    else:
                        not_found_count += 1
                        not_found_ids.append(source_id)
                    
                    if idx % 100 == 0 or idx == len(data):
                        print(f"  진행: {idx}/{len(data)} (매칭: {success_count}, 없음: {not_found_count}, 실패: {error_count})")
                    
                except Exception as e:
                    error_count += 1
                    error_msg = f"sourceId: {doc.get('sourceId')} - {str(e)[:80]}"
                    error_details.append(error_msg)
                    if error_count <= 5:
                        print(f"  [오류 {error_count}] {error_msg}")
            
            print("\n=== 커밋 시작 ===")
            conn.commit()
            print("✓ 커밋 완료")
            
            print(f"\n=== 결과 요약 ===")
            print(f"✓ 매칭 성공: {success_count}건")
            print(f"⚠ DB에 없음: {not_found_count}건")
            print(f"✗ 오류: {error_count}건")
            
            if not_found_ids:
                print(f"\n⚠ DB에서 못 찾은 source_id 샘플 (처음 20개):")
                for nf_id in not_found_ids[:20]:
                    print(f"  - {nf_id}")
                
                cursor.execute("""
                    SELECT MIN(source_id), MAX(source_id) 
                    FROM documents 
                    WHERE end_year = %s
                """, (end_year,))
                min_id, max_id = cursor.fetchone()
                print(f"\n📊 DB의 source_id 범위: {min_id} ~ {max_id}")
            
            if error_count > 5:
                print(f"\n처음 5개 외 {error_count - 5}개의 추가 오류가 발생했습니다.")
                print("모든 오류 보기:")
                for err in error_details[:20]:
                    print(f"  - {err}")
                if len(error_details) > 20:
                    print(f"  ... 외 {len(error_details) - 20}개 더")
            
        except pymysql.Error as e:
            print(f"\n❌ DB 오류 발생:")
            print(f"  Error Code: {e.args[0]}")
            print(f"  Error Message: {e.args[1]}")
            
        except Exception as e:
            print(f"\n❌ 일반 오류 발생: {e}")
            
        finally:
            if cursor:
                cursor.close()
            if conn:
                conn.close()
                print("\n✓ DB 연결 종료")


def main():
    base_path = r'C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html'
    end_year = 2015
    csv_file = '인사정보_부서코드추가.csv'
    
    parser = ApprovalDocParser(base_path, csv_file)
    
    print("HTML 파일 파싱 시작...")
    results = parser.process_all_files()
    
    output_json_path = 'update_drafter_activities.json'
    parser.save_to_json(results, output_json_path)
    
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'password': '1234',
        'database': 'any_approval'
    }
    
    print("\n" + "="*50)
    parser.save_to_mariadb_update(results, db_config, end_year)
    print("="*50)
    
    print(f"\n완료! 총 {len(results)}건 처리됨")


if __name__ == '__main__':
    main()

📊 CSV 조직도 로드 중... (인사정보_부서코드추가.csv)
✅ 총 156명 로드

HTML 파일 파싱 시작...
총 840개의 HTML 파일을 찾았습니다.
처리 중... [100/840] 20100225_연차휴가(3_2) 신청합니다._2002163.html
처리 중... [200/840] 20100401_[기술지원팀 격려 인센티브 지급품의]_2002268.html
처리 중... [300/840] 20100517_개인휴가품의_2002367.html
처리 중... [400/840] 20100705_파일서버 접근 권한 요청_2002467.html
처리 중... [500/840] 20100817_8월 아웃소싱 인건비 지급 품의-기술지원부문_2002570.html
처리 중... [600/840] 20100929_재직증명서 발급요청_2002666.html
처리 중... [700/840] 20101110_대전지사 공용 하드디스크 구매 요청_2002767.html
처리 중... [800/840] 20101221_12월 아웃소싱 인건비 지급 품의-전문가Gr.(김규일)_2002865.html
처리 중... [840/840] 20101231_법인카드 사용내역 정산 보고(10월,11월,12월)_2002907.html
JSON 파일 저장 완료: update_drafter_activities.json


=== DB 연결 시작 ===
Host: localhost, Database: any_approval
✓ DB 연결 성공 (FOUND_ROWS 모드)

=== 데이터 업데이트 시작 (840건) ===
  진행: 100/840 (매칭: 100, 없음: 0, 실패: 0)
  진행: 200/840 (매칭: 200, 없음: 0, 실패: 0)
  진행: 300/840 (매칭: 300, 없음: 0, 실패: 0)
  진행: 400/840 (매칭: 400, 없음: 0, 실패: 0)
  진행: 500/840 (매칭: 500, 없음: 0, 실패: 0)
  진행: 600/840 (매칭: 600, 없